In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from pathlib import Path
import camelot

In [2]:
pdf_path = "data/epa_scghg_report_draft_0.pdf"

In [3]:
tables = camelot.read_pdf(pdf_path, pages="87", flavor="stream")

In [4]:
df = tables[0].df.loc[4:].drop(labels=[5,6])

In [5]:
df

,0,1,2,3,4,5,6,7,8,9
4,,,SC-CO2,,,SC-CH4,,,SC-N2O,
7,,2.5%,2.0%,1.5%,2.5%,2.0%,1.5%,2.5%,2.0%,1.5%
8,Year,,,,,,,,,
9,2020,120,190,340,"1,300","1,600","2,300","35,000","54,000","87,000"
10,2030,140,230,380,"1,900","2,400","3,200","45,000","66,000","100,000"
11,2040,170,270,430,"2,700","3,300","4,200","55,000","79,000","120,000"
12,2050,200,310,480,"3,500","4,200","5,300","66,000","93,000","140,000"
13,2060,230,350,530,"4,300","5,100","6,300","76,000","110,000","150,000"
14,2070,260,380,570,"5,000","5,900","7,200","85,000","120,000","170,000"
15,2080,280,410,600,"5,800","6,800","8,200","95,000","130,000","180,000"


In [6]:
dcr_list = [2.5, 2, 1.5]
emit_list = ["CO2", "CH4", "N2O"]

In [7]:
df = df.set_index(0)

In [8]:
df.columns = pd.MultiIndex.from_product([emit_list, dcr_list])

In [9]:
df.reset_index().drop(labels=[0,1,2]).set_index(0)

CO2              CH4                   N2O                  
      2.5  2.0  1.5    2.5    2.0    1.5     2.5      2.0      1.5
0                                                                 
2020  120  190  340  1,300  1,600  2,300  35,000   54,000   87,000
2030  140  230  380  1,900  2,400  3,200  45,000   66,000  100,000
2040  170  270  430  2,700  3,300  4,200  55,000   79,000  120,000
2050  200  310  480  3,500  4,200  5,300  66,000   93,000  140,000
2060  230  350  530  4,300  5,100  6,300  76,000  110,000  150,000
2070  260  380  570  5,000  5,900  7,200  85,000  120,000  170,000
2080  280  410  600  5,800  6,800  8,200  95,000  130,000  180,000

In [13]:
scenarios = os.listdir('results/fy25')



In [15]:
files_list = glob('results/fy25/*/emit_irt.csv')


In [16]:
ex_file = pd.read_csv(files_list[0])
ex_file.head()

,eall,i,r,t,Value
0,CO2,biopower,p1,2010,841.765219
1,CO2,biopower,p3,2010,3503.640307
2,CO2,biopower,p3,2015,942.978960
3,CO2,biopower,p3,2020,942.978960
4,CO2,biopower,p3,2023,942.978960


In [18]:
df = ex_file.drop(columns=['i']).groupby(['eall','r','t']).sum().reset_index()

In [19]:
scc = pd.read_excel("data/scc_mult.xlsx")


In [26]:
# Ensure column names are consistent (case-insensitive match)
scc['eall'] = scc['eall'].ffill().str.upper()
# Merge on 'eall' and 't' columns
merged = pd.merge(
    df,
    scc,
    on=['eall', 't'],
    suffixes=('_emit', '_scc')
)


In [27]:
merged

,eall,r,t,Value_emit,Value_scc
0,CH4,p1,2010,379159.375093,800
1,CH4,p1,2015,153947.716824,1200
2,CH4,p1,2020,469555.535016,1600
3,CH4,p1,2023,225601.573111,1840
4,CH4,p1,2026,437163.347902,2080
...,...,...,...,...,...
4568,N2O,z122,2038,156073.151009,76400
4569,N2O,z122,2041,153442.125717,80400
4570,N2O,z122,2044,145725.467786,84600
4571,N2O,z122,2047,111285.169105,88800


In [25]:
scc['eall'].ffill()

0     CO2
1     CO2
2     CO2
3     CO2
4     CO2
5     CO2
6     CO2
7     CO2
8     CO2
9     CO2
10    CO2
11    CO2
12    CH4
13    CH4
14    CH4
15    CH4
16    CH4
17    CH4
18    CH4
19    CH4
20    CH4
21    CH4
22    CH4
23    CH4
24    N2O
25    N2O
26    N2O
27    N2O
28    N2O
29    N2O
30    N2O
31    N2O
32    N2O
33    N2O
34    N2O
35    N2O
Name: eall, dtype: object

In [28]:
df['t'].unique()

array([2010, 2015, 2020, 2023, 2026, 2029, 2032, 2035, 2038, 2041, 2044,
       2047, 2050])

In [29]:
scc['t'].unique()

array([2010, 2015, 2020, 2023, 2026, 2029, 2032, 2038, 2041, 2044, 2047,
       2050])

In [ ]:

# Multiply the values
merged['product'] = merged['Value_emit'] * merged['Value_scc']
